In [1]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
conn = sqlite3.connect("ss.db")
cur  = conn.cursor()

import time
import os

import pandas as pd

In [2]:
#If downloads table does not exist, create one 

#Uncomment to delete downloads table, must restart downloads if that happens
#cur.execute('''DROP TABLE downloads''')

try:
    qry = """SELECT bouncepoints_full.ID_Full, 
    bouncepoints_full.Event, 
    bouncepoints_full.Station, 
    bouncepoints_full.Network, 
    bouncepoints_full.Range, 
    bouncepoints_full.MidLat, 
    bouncepoints_full.MidLon, 
    bouncepoints_full.EvtTime, 
    bouncepoints_full.BAzimuth, 
    downloads.Success
    FROM bouncepoints_full 
    LEFT JOIN downloads 
    ON bouncepoints_full.ID_Full=downloads.ID_Full"""
    
    dfds = pd.read_sql_query(qry, conn, index_col=["ID_Full"])
    dfds = dfds.query('MidLat > 30 and MidLat < 75 and MidLon < -60 and MidLon > -135 and Success != "True"')
    
except:
    dfds = pd.DataFrame(columns=["ID_Full","Attempted","Success","Failure_Reason"])
    dfds.to_sql('downloads',conn, dtype = {"ID_Full" : 'TEXT',
                               "Attempted" : "BOOLEAN",          
                               "Success" : "BOOLEAN",
                               "Failure_Reason" : "TEXT"})
    
    dfds = pd.read_sql_query("SELECT ID_Full, Event, Station, Network, Range, MidLat, MidLon, EvtTime, BAzimuth from bouncepoints_full", conn, index_col=["ID_Full"])
    dfds = dfds.query('MidLat > 30 and MidLat < 75 and MidLon < -60 and MidLon > -135')
    
dfds

,Event,Station,Network,Range,MidLat,MidLon,EvtTime,BAzimuth,Success
ID_Full,,,,,,,,,
C201401040011A.II.ALE,C201401040011A,ALE,II,103.074093,30.927075,-69.924458,2014-01-04T00:11:55.400000Z,188.301250,None
C201401040011A.IU.ULN,C201401040011A,ULN,IU,152.769263,55.670139,-66.064350,2014-01-04T00:11:55.400000Z,356.027537,None
C201401040011A.IU.YAK,C201401040011A,YAK,IU,136.232722,45.428759,-89.371913,2014-01-04T00:11:55.400000Z,28.373276,None
C201401040011A.IU.YSS,C201401040011A,YSS,IU,142.149065,35.731515,-116.773406,2014-01-04T00:11:55.400000Z,57.404668,None
C201401050336A.AK.ANM,C201401050336A,ANM,AK,85.281008,41.963592,-99.897879,2014-01-05T03:36:42.500000Z,89.099623,None
C201401050336A.AK.ATKA,C201401050336A,ATKA,AK,90.991399,38.380774,-110.353076,2014-01-05T03:36:42.500000Z,81.104278,None
C201401050336A.AK.BAGL,C201401050336A,BAGL,AK,74.041600,36.543203,-97.219937,2014-01-05T03:36:42.500000Z,109.413648,None
C201401050336A.AK.BAL,C201401050336A,BAL,AK,74.341225,36.837260,-97.023918,2014-01-05T03:36:42.500000Z,109.327216,None
C201401050336A.AK.BARK,C201401050336A,BARK,AK,74.200891,36.554333,-97.371114,2014-01-05T03:36:42.500000Z,109.021977,None


In [3]:
from obspy import UTCDateTime

from obspy.taup import TauPyModel

from obspy.clients.fdsn import Client

client = Client("IRIS")

model = TauPyModel('ak135')



#def save_error_dataframe(dataframe):
#    dfe.to_csv('errors.csv')
    
def insert_database_info(ID_Full):
    cur.execute("INSERT INTO downloads (ID_Full) VALUES ('%s')" % ID_Full)
    

def update_database_error(error_msg,ID_Full):
    cur.execute("UPDATE downloads SET Failure_Reason = '%s' WHERE ID_Full = '%s'" % (error_msg, ID_Full ) )
    cur.execute("UPDATE downloads SET Attempted = 'True' WHERE ID_Full = '%s'" % (ID_Full ) )
    
def update_database_status(success,ID_Full):
    cur.execute("UPDATE downloads SET Success = '%s' WHERE ID_Full = '%s'" % (success, ID_Full ) )
    cur.execute("UPDATE downloads SET Attempted = 'True' WHERE ID_Full = '%s'" % (ID_Full ) )

for bp in dfds.iterrows():
    etime = UTCDateTime(bp[1].EvtTime)
    sta = bp[1].Station
    net = bp[1].Network
    deldeg = bp[1].Range
    
    print("Working on    " + bp[0] )
    insert_database_info(bp[0])
    
    arrivals = model.get_travel_times(0.0, deldeg, phase_list = ['SS'])
    
    
    time_ss = arrivals[0].time
    
    t1 = etime + time_ss - 200
    t2 = etime + time_ss + 200
    
    clock=time.time()
    try:
        st = client.get_waveforms(net, sta, "*", "LH*", t1, t2, attach_response = True)
    except:
        err = 'No Data at %s.%s' % (net, sta)
        #print(err)
        #dfe.loc[bp[0],"Status"] = err
        update_database_error(err, bp[0])
        continue
    print('  step completed in %8.2e s' % (time.time() - clock) )
        
    

    try:
        st.remove_response(output='ACC', pre_filt=(0.005, 0.01, 0.5, 1.0))
    except ValueError as e:
        err = 'Response Error: %s' % e
        #print(err)
        #dfe.loc[bp[0],"Status"] = err
        update_database_error(err, bp[0])
    
    try:
        inv = client.get_stations(
            starttime=t1, endtime=t2, 
            network=net, sta=sta, 
            loc="*", channel="LH*", level="response")
        
        st.rotate('->ZNE', inventory = inv)
        st.rotate('NE->RT', back_azimuth = bp[1].BAzimuth)
    except ValueError as e:
        err = 'Rotation Error: %s' % e
        #print(err)
        #dfe.loc[bp[0],"Status"] = err
        update_database_error(err, bp[0])
        
        continue
    
    bpid = bp[0]
    
    directory = 'data/mseed/%s/' % (bp[1].Event)
    
    filename = directory + bpid + '.mseed'
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    trT = st[0]
    
    for tr in st:
        if 'T' in tr.stats.channel:
            trT = tr
    
    trT.resample(1)
    
    trT.write(filename)
    
    #dfe.loc[bp[0],"Status"] = 'OK'
    update_database_status(True, bp[0])
    
    #save_error_dataframe(dfe)
    conn.commit()
    


Working on    C201401040011A.II.ALE
Working on    C201401040011A.IU.ULN
  step completed in 2.02e+00 s
Working on    C201401040011A.IU.YAK
  step completed in 1.09e+00 s
Working on    C201401040011A.IU.YSS
  step completed in 1.13e+00 s
Working on    C201401050336A.AK.ANM
Working on    C201401050336A.AK.ATKA
Working on    C201401050336A.AK.BAGL
Working on    C201401050336A.AK.BAL
Working on    C201401050336A.AK.BARK
Working on    C201401050336A.AK.BARN
Working on    C201401050336A.AK.BCP
Working on    C201401050336A.AK.BERG
Working on    C201401050336A.AK.BESE
Working on    C201401050336A.AK.BGLC
Working on    C201401050336A.AK.BMR
Working on    C201401050336A.AK.BPAW
Working on    C201401050336A.AK.BRLK
Working on    C201401050336A.AK.BRSE
Working on    C201401050336A.AK.BWN
Working on    C201401050336A.AK.CAST
Working on    C201401050336A.AK.CCB
Working on    C201401050336A.AK.CHI
Working on    C201401050336A.AK.CHN
Working on    C201401050336A.AK.CNP
Working on    C201401050336A.AK.

Working on    C201401072302A.AK.BWN
Working on    C201401072302A.AK.CAST
Working on    C201401072302A.AK.CCB
Working on    C201401072302A.AK.COLD
Working on    C201401072302A.AK.DHY
Working on    C201401072302A.AK.DOT
Working on    C201401072302A.AK.FYU
Working on    C201401072302A.AK.GAMB
Working on    C201401072302A.AK.GHO
Working on    C201401072302A.AK.HDA
Working on    C201401072302A.AK.KTH
Working on    C201401072302A.AK.MCK
Working on    C201401072302A.AK.MDM
Working on    C201401072302A.AK.MLY
Working on    C201401072302A.AK.NEA
Working on    C201401072302A.AK.PAX
Working on    C201401072302A.AK.PPD
Working on    C201401072302A.AK.PPLA
Working on    C201401072302A.AK.RDOG
Working on    C201401072302A.AK.RIDG
Working on    C201401072302A.AK.RND
Working on    C201401072302A.AK.SAW
Working on    C201401072302A.AK.SCM
Working on    C201401072302A.AK.SCRK
Working on    C201401072302A.AK.SKN
Working on    C201401072302A.AK.SSN
Working on    C201401072302A.AK.TNA
Working on    C201401

Working on    C201401072302A.SL.CRNS
Working on    C201401072302A.SL.DOBS
Working on    C201401072302A.SL.GBAS
Working on    C201401072302A.SL.GBRS
Working on    C201401072302A.SL.GCIS
Working on    C201401072302A.SL.GOLS
Working on    C201401072302A.SL.GORS
Working on    C201401072302A.SL.GROS
Working on    C201401072302A.SL.JAVS
Working on    C201401072302A.SL.KNDS
Working on    C201401072302A.SL.KOGS
Working on    C201401072302A.SL.LJU
Working on    C201401072302A.SL.MOZS
Working on    C201401072302A.SL.PDKS
Working on    C201401072302A.SL.PERS
Working on    C201401072302A.SL.ROBS
Working on    C201401072302A.SL.SKDS
Working on    C201401072302A.SL.VISS
Working on    C201401072302A.SL.VNDS
Working on    C201401072302A.SL.VOJS
Working on    C201401072302A.SL.ZALS
Working on    C201401072302A.TA.A36M
  step completed in 1.33e+00 s
Working on    C201401072302A.TA.C36M
  step completed in 1.22e+00 s
Working on    C201401072302A.TA.EPYK
  step completed in 1.34e+00 s
Working on    C20140

  step completed in 1.58e+00 s
Working on    C201401111310A.EI.VAL
Working on    C201401111310A.G.IVI
  step completed in 1.36e+00 s
Working on    C201401111310A.GE.DAG
  step completed in 1.27e+00 s
Working on    C201401111310A.GE.DSB
Working on    C201401111310A.GE.KBS
Working on    C201401111310A.GE.PUL
Working on    C201401111310A.GE.SUMG
  step completed in 1.44e+00 s
Working on    C201401111310A.GE.VAL
Working on    C201401111310A.IC.HIA
  step completed in 1.99e+00 s
Working on    C201401111310A.II.ALE
Working on    C201401111310A.II.ARU
  step completed in 1.31e+00 s
Working on    C201401111310A.II.BORG
  step completed in 2.18e+00 s
Working on    C201401111310A.II.BRVK
  step completed in 2.22e+00 s
Working on    C201401111310A.II.CMLA
  step completed in 2.26e+00 s
Working on    C201401111310A.II.ESK
  step completed in 2.32e+00 s
Working on    C201401111310A.II.KDAK
  step completed in 2.17e+00 s
Working on    C201401111310A.II.KURK
  step completed in 2.07e+00 s
Working on 

Working on    C201401130401A.IU.PET
  step completed in 1.32e+00 s
Working on    C201401130401A.IU.TIXI
  step completed in 1.24e+00 s
Working on    C201401130401A.IU.WAKE
  step completed in 2.11e+00 s
Working on    C201401130401A.IU.YAK
  step completed in 1.15e+00 s
Working on    C201401130401A.IU.YSS
  step completed in 1.14e+00 s
Working on    C201401130401A.NV.KEMF
Working on    C201401130401A.NV.NC27
Working on    C201401130401A.NV.NC89
Working on    C201401130401A.NV.NCBC
Working on    C201401130401A.NV.NCHR
Working on    C201401130401A.PB.B012
  step completed in 1.25e+00 s
Working on    C201401130401A.PB.B928
  step completed in 1.19e+00 s
Working on    C201401130401A.TA.A36M
  step completed in 1.57e+00 s
Working on    C201401130401A.TA.C36M
  step completed in 1.27e+00 s
Working on    C201401130401A.TA.EPYK
  step completed in 1.26e+00 s
Working on    C201401130401A.TA.HDA
  step completed in 1.35e+00 s
Working on    C201401130401A.TA.POKR
  step completed in 2.18e+00 s
Wor

...

  step completed in 1.35e+00 s
Working on    C201401160733A.TA.H45A
  step completed in 1.26e+00 s
Working on    C201401160733A.TA.H46A
  step completed in 1.21e+00 s
Working on    C201401160733A.TA.H47A
  step completed in 1.32e+00 s
Working on    C201401160733A.TA.H48A
  step completed in 1.20e+00 s
Working on    C201401160733A.TA.H52A
  step completed in 1.41e+00 s
Working on    C201401160733A.TA.H53A
  step completed in 1.25e+00 s
Working on    C201401160733A.TA.H55A
  step completed in 1.33e+00 s
Working on    C201401160733A.TA.H56A
  step completed in 1.26e+00 s
Working on    C201401160733A.TA.H57A
  step completed in 1.26e+00 s
Working on    C201401160733A.TA.H58A
  step completed in 1.40e+00 s
Working on    C201401160733A.TA.H59A
  step completed in 1.26e+00 s
Working on    C201401160733A.TA.H60A
  step completed in 1.21e+00 s
Working on    C201401160733A.TA.H61A
  step completed in 1.29e+00 s
Working on    C201401160733A.TA.H62A
  step completed in 1.28e+00 s
Working on    C20

  step completed in 1.21e+00 s
Working on    C201401160733A.TA.N60A
  step completed in 1.39e+00 s
Working on    C201401160733A.TA.N61A
  step completed in 1.20e+00 s
Working on    C201401160733A.TA.N62A
  step completed in 1.25e+00 s
Working on    C201401160733A.TA.N63A
  step completed in 1.23e+00 s
Working on    C201401160733A.TA.O48A
  step completed in 1.26e+00 s
Working on    C201401160733A.TA.O49A
  step completed in 1.22e+00 s
Working on    C201401160733A.TA.O50A
  step completed in 1.27e+00 s
Working on    C201401160733A.TA.O51A
  step completed in 1.34e+00 s
Working on    C201401160733A.TA.O52A
  step completed in 1.41e+00 s
Working on    C201401160733A.TA.O53A
  step completed in 1.32e+00 s
Working on    C201401160733A.TA.O54A
  step completed in 1.25e+00 s
Working on    C201401160733A.TA.O55A
  step completed in 1.27e+00 s
Working on    C201401160733A.TA.O56A
  step completed in 1.21e+00 s
Working on    C201401160733A.TA.O57A
  step completed in 1.20e+00 s
Working on    C20

  step completed in 1.25e+00 s
Working on    C201401160733A.TA.W52A
  step completed in 1.13e+00 s
Working on    C201401160733A.TA.W53A
  step completed in 1.19e+00 s
Working on    C201401160733A.TA.W54A
  step completed in 1.14e+00 s
Working on    C201401160733A.TA.W56A
  step completed in 1.18e+00 s
Working on    C201401160733A.TA.W57A
  step completed in 1.20e+00 s
Working on    C201401160733A.TA.W58A
  step completed in 1.21e+00 s
Working on    C201401160733A.TA.W59A
  step completed in 1.20e+00 s
Working on    C201401160733A.TA.W60A
  step completed in 1.19e+00 s
Working on    C201401160733A.TA.W61A
  step completed in 1.19e+00 s
Working on    C201401160733A.TA.WHTX
  step completed in 1.26e+00 s
Working on    C201401160733A.TA.X40A
  step completed in 1.21e+00 s
Working on    C201401160733A.TA.X43A
  step completed in 1.33e+00 s
Working on    C201401160733A.TA.X51A
  step completed in 1.14e+00 s
Working on    C201401160733A.TA.X52A
  step completed in 1.26e+00 s
Working on    C20

Working on    C201401160733A.XO.KH44
  step completed in 1.66e+00 s
Working on    C201401160733A.XO.KH46
  step completed in 1.57e+00 s
Working on    C201401160733A.XO.KH48
  step completed in 1.39e+00 s
Working on    C201401160733A.XO.KH49
  step completed in 1.48e+00 s
Working on    C201401160733A.XO.KI25
  step completed in 1.56e+00 s
Working on    C201401160733A.XO.KI27
  step completed in 1.45e+00 s
Working on    C201401160733A.XO.KI29
  step completed in 1.45e+00 s
Working on    C201401160733A.XO.KI31
  step completed in 1.46e+00 s
Working on    C201401160733A.XO.KI33
  step completed in 1.56e+00 s
Working on    C201401160733A.XO.KI35
  step completed in 1.62e+00 s
Working on    C201401160733A.XO.KI37
Working on    C201401160733A.XO.KI39
  step completed in 1.53e+00 s
Working on    C201401160733A.XO.KI41
  step completed in 1.50e+00 s
Working on    C201401160733A.XO.KI43
  step completed in 1.49e+00 s
Working on    C201401160733A.XO.KI45
  step completed in 1.57e+00 s
Working on 

  step completed in 1.54e+00 s
Working on    C201401261039A.PM.PMOZ
  step completed in 1.57e+00 s
Working on    C201401261039A.PM.PVAQ
  step completed in 9.99e-01 s
Working on    C201401261039A.PM.ROSA
  step completed in 1.54e+00 s
Working on    C201401261039A.SS.COI
  step completed in 1.01e+00 s
Working on    C201401261355A.G.PPTF
  step completed in 1.33e+00 s
Working on    B201401271642A.AK.ANM
Working on    B201401271642A.AK.GAMB
Working on    B201401271642A.AK.RDOG
Working on    B201401271642A.AK.TNA
Working on    B201401271642A.AT.SMY
Working on    B201401271642A.G.INU
  step completed in 1.20e+00 s
Working on    B201401271642A.IC.BJT
  step completed in 1.99e+00 s
Working on    B201401271642A.IC.HIA
  step completed in 2.04e+00 s
Working on    B201401271642A.IC.MDJ
  step completed in 2.10e+00 s
Working on    B201401271642A.IC.SSE
  step completed in 1.95e+00 s
Working on    B201401271642A.II.ALE
Working on    B201401271642A.II.ERM
  step completed in 2.18e+00 s
Working on  

In [4]:
conn.close()